In [1]:
import numpy as np
import xarray as xr
import pandas as pd
import netCDF4 as nc
import matplotlib.pyplot as plt
import sys
import numpy as np
import matplotlib as mpl
from mpl_toolkits.basemap import Basemap
import time
from datetime import datetime
from scipy import interpolate
import cartopy.crs as ccrs
import pyroms

GRID

In [2]:
mask_url = 'https://dap.ceda.ac.uk/thredds/dodsC/bodc/UOX220077/WINDS-M/supplementary_files/croco_grd.nc'
ds_mask = xr.open_dataset(mask_url)

In [3]:
clipped_mask_one = ds_mask.sel(eta_rho=slice(680, 986), xi_rho=slice(150, 451),
                               eta_psi=slice(680, 986), xi_psi=slice(150, 451),
                               eta_u=slice(680, 986), xi_u=slice(150, 451),
                               eta_v=slice(680, 986), xi_v=slice(150, 451)) # clipping the domain

clipped_mask = clipped_mask_one.drop_vars(['xl','el','depthmin','depthmax', 'spherical', 'angle', 'h', 'hraw', 'alpha',
                                 'f','pm','pn','dndx','dmde','x_rho','x_u','x_v','x_psi','y_rho',
                                  'y_v','y_u', 'y_psi',   
                                ])

In [5]:
clipped_mask.to_netcdf('/mnt/iscsi2/OceanParcels/InputObsDom/WINDS_as_NEMO_mesh_prof.nc')

DATA HALF HOUR VELOCITY FOR THE CASE OF APRIL

In [64]:
base_url = 'https://dap.ceda.ac.uk/thredds/dodsC/bodc/UOX220077/WINDS-M/1999/WINDS-M_SFC_1999.nc'
ds_complete = xr.open_dataset(base_url)

In [65]:
# Dropping unnecessary variables
dropping_variables = ds_complete.drop_vars(['s_w','hc','s_rho',
                                 'theta_s','theta_b','Tcline','Vtransform','h','f','pm','pn','lon_rho','lat_rho',
                                  'angle', 'Cs_r','sc_r','Cs_w','lon_u','lon_v','lat_u', 'lat_v',
                                  'sc_w'])

# Clipping considering only the domain of interest for the thesis case study
clipped_domain = dropping_variables.sel(y_rho=slice(680, 986), x_rho=slice(150, 451),
                          y_u=slice(680, 986), x_u=slice(150, 451),
                          y_v=slice(680, 986), x_v=slice(150, 451))

# Considering only 70 + 30 days from 1st April (so 100*24*2+1) values of 'APRIL', so from 01-04-1997 to 09-07-1997, so from 4320 to 4320+4801
days100_from_april_1999 = clipped_domain.isel(time_counter=slice(4320, 4320+4801))

In [66]:
days100_from_april_1999_velocity_u = days100_from_april_1999.drop_vars(['nav_lon_v', 'nav_lat_v', 'v_surf', 'time'])
days100_from_april_1999_velocity_v = days100_from_april_1999.drop_vars(['nav_lon_u', 'nav_lat_u', 'u_surf', 'time'])

In [67]:
chunks = {'time_counter': 100, 'y_u': 306, 'x_u': 301}
days100_from_april_1999_velocity_u_chunked = days100_from_april_1999_velocity_u.chunk(chunks)
days100_from_april_1999_velocity_u_chunked.to_netcdf("/mnt/iscsi2/OceanParcels/InputObsDom/days100_from_april_1999_velocity_u_OK.nc")

In [68]:
chunks = {'time_counter': 100, 'y_v': 306, 'x_v': 301}
days100_from_april_1999_velocity_v_chunked = days100_from_april_1999_velocity_v.chunk(chunks)
days100_from_april_1999_velocity_v_chunked.to_netcdf("/mnt/iscsi2/OceanParcels/InputObsDom/days100_from_april_1999_velocity_v_OK.nc")